<h1><center> TP2: Modèles linéaires<br>
    Perceptron, Adaline, Regression Logistique et SVM</center></h1>

Nous allons nous intéresser à l'implémentation des modèles du perceptron ('p'), de l'Adline ('a') , de la regression logistique ('r') et des SVM ('s') dont les pseudo-codes puvent être résumés comme suit:

input: Train, eta, m, MaxEp, modele

init : w

epoque=0

while epoque<=MaxEp

    err=0

    for i in 1:m

        choisir un exemple (x,y) de Train de façon aléatoire

        h <- w*x

        if((modele = 'a') or (modele = 'r'))

            w <- w - eta*grad(L(w))

        elif((modele = 'p') and (y*h <= 0))

            w <- w - eta*grad(L(w))

        elif((modele = 's') and (1-y*h >= 0))

            w <- w - eta*grad(L(w))

     epoque <- epoque+1

output: w


Pour une fonction de prédiction $h_\mathbf{w}(\mathbf{x})=w_0+\langle \mathbf{w},\mathbf{x}\rangle$; $\mathcal{L}(\mathbf{w})$ est la fonction de coût qui dans le cas de 
- Perceptron: $\mathcal{L}(\mathbf{w})=(-y*h_\mathbf{w}(\mathbf{x}))$,
- Adaline: $\mathcal{L}(\mathbf{w})=(y-h_\mathbf{w}(\mathbf{x}))^2$,
- Régression logistique: $\mathcal{L}(\mathbf{w})=\log(1+e^{-yh_\mathbf{w}(\mathbf{x})})$,
- SVM: $\mathcal{L}(\mathbf{w})=\max(0,1-y*h_\mathbf{w}(\mathbf{x}))$

$grad(\mathcal{L}(\mathbf{w}))$ est le gradient de la fonction $\mathcal{L}(\mathbf{w})$ dont on a vu les expressions pour ces modèles en TD.


<font color='red'><b>Question 1:</b></font> Créer une liste de 4 éléments correspondant à l'exemple ET logique; chaque élément de la liste est une liste dont la dernière caractéristique est la classe de l'exemple et les premières caractéristiques leurs coordonnées.

In [101]:
import numpy as np

#ET est notre exemple
#ici le -1 c'est le 0 si on avait raisonné en binaire
ET=np.array([[0, 0, -1], [0, 1, -1], [1, 0, -1], [1, 1, 1]])

In [102]:
print(ET)

[[ 0  0 -1]
 [ 0  1 -1]
 [ 1  0 -1]
 [ 1  1  1]]


<font color='red'><b>Question 2:</b></font> Coder le programme du Perceptron (modele = 'p').

Indication: Vous pouvez écrire une fonction qui calcule le produit scalaire entre un exemple $\mathbf{x}=(x_1,\ldots,x_d)$ et le vecteur poids $\mathbf{w}=(w_0,w_1,\ldots,w_d)$:
$\langle \mathbf{w},\mathbf{x} \rangle=w_0+\sum_{j=1}^d w_j x_j$; et une autre qui pour une matrice de données qui n'ont pas servi à apprendre le vecteur poids $\mathbf{w}$ (i.e. la base Test), calcule le taux d'erreur sur cette base.

In [103]:
'''w=(3,) et x=(4,2) Le produit scalaire est possible car x est un vecteur ligne et w est un vecteur colonne.
w[1:]done (2,) et x[:,1:]done (4,2) donc le nombre de ligne de colonne de x[:,1:] est le même que celui de w[1:]

'''

'''Pour calculer le taux d'erreur, on va diviser le nombre de fois où le produit scalaire est négatif par la taille du
 de l'exemple x.En gros on fait la moyenne des exemples mal classés par rapport a la vrai sortie de l'exemple
'''
class LinearModel:
    def __init__(self, N, eta):
    #N matrix size, eta learning rate
        self.w = np.zeros(N)
        self.eta = eta

    def h(self, x):
        # Je veux avoir tous w sauf la 1er case(w est un array numpy et non une liste)
        # v2:w[0]+sum(Xj*Wj for (Xj,Wj) in zip(x,w[1:]))
        return self.w[0] + np.dot(x, self.w[1:])

    # def model(self, x):
    #     return 1 if (self.produit_scalaire(x) >= 0) else 0

    # def prediction(self, X):
    #     pred = []
    #     for x in X:
    #         result = self.model(x)
    #         pred.append(result)
    #     return np.array(pred)

    def taux_erreur(self, Train):
        nb_neg = 0
        x = Train[:, :-1]
        y = Train[:, -1]
        for X, Y in zip(x, y):
            # si l'exemple est mal classé
            if Y * self.h(X) <= 0:
                nb_neg += 1
        return nb_neg / x.shape[0]
    
    def TAUX_ERREUR(self, Test):
        nb_mal_classes = 0
        X_test=Test[:,:-1]
        Y_test=Test[:,-1]
        for x, y in zip(X_test, Y_test):
                nb_mal_classes =((self.h(x)*y)<=0).sum()
        return nb_mal_classes / len(X_test)


 
class Perceptron(LinearModel):
    def fit(self, Train, m, MaxEp):
        w = np.zeros(Train.shape[1])
        for epoque in range(MaxEp):
            # pour chaque exemple dans Train
            for ligne in range(m):
                # y est la sortie associé à l'exemple x[ligne].Elle se trouve sur la dernière colonne de x qui est la classe de l'exemple
                x = Train[ligne, :-1]
                y = Train[ligne, -1]
                if y * self.h(x) <= 0:
                    self.w[0] = self.w[0] + self.eta * y
                    self.w[1:] = self.w[1:] + self.eta * y * x
        return self.w


<font color='red'><b>Question 3:</b></font> Appliquer le programme du perceptron sur la base du ET logique, calculer le taux d'erreur du modèle sur cette base.


In [104]:


'''
"Train=base de données d'entraînement qui contient des exemples pour apprendre un modèle.

Epoque=nombre de cycles d'apprentissage effectués.

MaxEp=nombre maximum d'itérations pour l'algorithme, ce qui signifie que l'apprentissage s'arrêtera si le nombre d'itérations atteint MaxEp.

m=nombre d'exemples dans Train.
'''

# Definition du pas d'apprentissage
eta=0.1
# Nombre d'itérations (époques)
MaxEp = 500

perceptron = Perceptron(len(ET[0]), eta)
print(perceptron.fit(ET,ET.shape[0],MaxEp))
print("taux d'erreur pour perceptrion ", perceptron.taux_erreur(ET))


[-0.2  0.2  0.1]
taux d'erreur pour perceptrion  0.0


<font color='red'><b>Question 4:</b></font> Coder les modèles Adaline ('a'), Régression Logistique ('r') et SVM ('s') et reporter leur erreur sur le problème jouet de ET logique.


In [105]:

w_a=np.zeros(ET.shape[1]) 
w_r=np.zeros(ET.shape[1]) 
w_s=np.zeros(ET.shape[1]) 

class Adaline(LinearModel):
    def fit(self, Train, m, MaxEp):
        for epoque in range(MaxEp):
            for ligne in range(m):
                y=Train[ligne,-1]
                x=Train[ligne,:-1]
                res = self.h(x)
                self.w[0]=self.w[0]+self.eta*(y-res)
                self.w[1:]=self.w[1:]+self.eta*(y-res)*x
        return self.w
    
class LogisticRegression(LinearModel):
    def fit(self, Train, m, MaxEp):
        for epoque in range(MaxEp):
            for ligne in range(m):
                y=Train[ligne,-1]
                x=Train[ligne,:-1]
                L=(1-(1/(1+np.exp(-y*self.h(x)))))
                self.w[0]=self.w[0]+self.eta*y*L
                self.w[1:]=self.w[1:]+self.eta*L*x
        return self.w
        
class SVM(LinearModel):
    def fit(self, Train, m, MaxEp):
        for epoque in range(MaxEp):
            for ligne in range(m):
                y=Train[ligne,-1]
                x=Train[ligne,:-1]
                if y*self.h(x)<=1:
                    self.w[0]=self.w[0]+self.eta*y
                    self.w[1:]=self.w[1:]+self.eta*y*x
        return self.w



adaline =Adaline(len(ET[0]), eta)
print(adaline.fit(ET,ET.shape[0],MaxEp))
print("taux d'erreur pour perceptrion ", adaline.taux_erreur(ET))
        

regression_logistique = LogisticRegression(len(ET[0]), eta)
print(regression_logistique.fit(ET,ET.shape[0],MaxEp))
print("taux d'erreur pour perceptrion ", regression_logistique.taux_erreur(ET))

svm = SVM(len(ET[0]), eta)
print(svm.fit(ET,ET.shape[0],MaxEp))
print("taux d'erreur pour perceptrion ", svm.taux_erreur(ET))


[-1.55555556  1.11111111  1.05555556]
taux d'erreur pour perceptrion  0.0
[-12.89999094   6.67033192  11.19043712]
taux d'erreur pour perceptrion  0.0
[-3.2  2.2  2.1]
taux d'erreur pour perceptrion  0.0


<font color='red'><b>Question 5:</b></font> Nous allons maintenant nous intéresser au comportement de ces modèles sur la base sonar de la collection UCI (http://archive.ics.uci.edu/ml/index.php). Cette base contient 208 exemples en dimension 60 séparés par `,` et la dernière élément correspond à la classe de l'exemple.

    1. Télécharger la collection avec la fonction read_table de la librairie pandas (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_table.html). Les options nécessaires sont `sep=','` et `header=None`  
    2. Créer une liste de listes correspondant à la collection; pour cela initialiser la première liste et en parcourant chaque ligne de la matrice de données; créer une liste associée en remplaçant le dernier élément par `-1` ou `+1` et insérer la dans la première liste. 
    Indication: Utiliser la fonction `loc`. 
    3. Scinder la liste en deux listes `x_train` (75%) and `x_test` (25%) en la mélangeant aléatoirement au préalable. 
    Indication: Utiliser les fonctions `shuffle` de la librairie `random` et `train_test_split` de la librairie `sklearn.model_selection`  
    

In [106]:

import pandas as pd
import sklearn.model_selection as sk
from random import shuffle
from sklearn.model_selection  import train_test_split

data = pd.read_table('http://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data', sep=',', header=None)

#Initialisation de la liste principale
collection = []
#On parcourt les lignes du dataframe
for i in range(data.shape[0]):
    ligne_collection=data.iloc[i].tolist()
    #data.iloc[:,-1] = data.iloc[:,-1].replace('R',1).replace('M',-1) ou data[60] = np.where(data[60]=='R', 1, -1)
    if ligne_collection[-1]=='R':
       ligne_collection[-1]=1
    else:
        ligne_collection[-1]=-1
    collection.append(ligne_collection)

#Mélanger au préalable la colletion
shuffle(collection)

collection=np.array(collection)
X=collection[:,:-1]
Y=collection[:,-1]

'''
On va diviser la collection en un ensemble d'entraînement et un ensemble de test, pour pouvoir évaluer la
performance du modèle sur des données qu'il n'a jamais vues pendant l'entraînement(qui n'a pas servi a apprendre le modéle)
'''
#On divise la collection en 2 parties:une pour l'entrainement(75%) et une pour le test(25%)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25)

<font color='red'><b>Question 6:</b></font> Appliquer ces modèles sur cette base en prenant comme $MaxEp=500$, le pas d'apprentissage $\eta=0.1$ et en choisissant les bases Train et Test de façon aléatoire; Reporter l'erreur moyenne de ces modèles obtenues sur les 20 bases Test? 

In [107]:

erreur_p =[]
erreur_a =[]
erreur_r =[]
erreur_s =[]
nb_base_tests=20

#A chaque itéaration,on aura des bases d'entrainement et de test différentes
for i in range(nb_base_tests):
    #Générer une base d'entrainement et une base de test de façon aléatoire grace a train_test_split
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25)

    Test= np.column_stack((X_test, Y_test))
    Train= np.column_stack((X_train, Y_train))

    w_p=np.zeros(Train.shape[1])
    w_a=np.zeros(Train.shape[1])
    w_r=np.zeros(Train.shape[1])
    w_s=np.zeros(Train.shape[1])

    #On initialise les modeles
    perceptron = Perceptron(len(Train[0]), eta)
    adaline =Adaline(len(Train[0]), eta)
    regression_logistique = LogisticRegression(len(Train[0]), eta)
    svm= SVM(len(Train[0]), eta)
    
    # On entraine les modéles 
    w_p = perceptron.fit(Train,Train.shape[0],MaxEp)
    w_a = adaline.fit(Train,Train.shape[0],MaxEp)
    w_r = regression_logistique.fit(Train,Train.shape[0],MaxEp)
    w_s = svm.fit(Train,Train.shape[0],MaxEp)

    
    # Calculer l'erreur moyenne sur les données de test
    erreur_p.append(perceptron.TAUX_ERREUR(Test))
    erreur_a.append(adaline.TAUX_ERREUR(Test))
    erreur_r.append(regression_logistique.TAUX_ERREUR(Test))
    erreur_s.append(svm.TAUX_ERREUR(Test))
    

erreur_moyenne_p =np.mean(erreur_p)
erreur_moyenne_a =  np.mean(erreur_a)
erreur_moyenne_r = np.mean(erreur_r)
erreur_moyenne_s =np.mean(erreur_s)

print("Erreur moyenne du modèle Perceptron:", erreur_moyenne_p)
print("Erreur moyenne du modèle Adaline:", erreur_moyenne_a)
print("Erreur moyenne du modèle Regression Logistique:", erreur_moyenne_r)
print("Erreur moyenne du modèle SVM:", erreur_moyenne_s)

/tmp/ipykernel_13910/1455021576.py:32: RuntimeWarning: overflow encountered in exp
  L=(1-(1/(1+np.exp(-y*self.h(x)))))


Erreur moyenne du modèle Perceptron: 0.004807692307692308
Erreur moyenne du modèle Adaline: 0.0057692307692307696
Erreur moyenne du modèle Regression Logistique: 0.006730769230769232
Erreur moyenne du modèle SVM: 0.004807692307692308


  | Collection | Adaline            | Perceptron          | Régression Logistique| SVM                 | 
  |------------|--------------------|---------------------|----------------------|---------------------|
  |   SONAR    |0.006730769230769232|0.0028846153846153848|0.009615384615384614  |0.0028846153846153848|


Nous allons étudier l'impact de la nomralisation sur les prédictions. Pour cela nous considérons deux stratégies de normalisation communément utilisées dans la littérature:
* Stratégie <i>max</i>: consiste à normaliser chaque caractéristique du vecteur réprésentatif d'une observation par la valeur maximale de cette caractéristiques
* Stratégie <i>norme</i>: consiste à normaliser chaque caractéristique du vecteur réprésentatif d'une observation par la norme de ce vecteur.

Nous considérons ces trois autres collections de la base UCI:

        * https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29
        * https://archive.ics.uci.edu/ml/datasets/spambase
        * https://archive.ics.uci.edu/ml/datasets/ionosphere

<font color='red'><b>Question 7:</b></font> Ecrire une fonction qui prend en entrée la collection des données et qui retourne la collections normalisée suivant les stratégies <i>max</i> et <i>norme</i>. 
        

In [108]:
'''
Un vecteur=[....]donc les observations sont les lignes de la collection et les caractéristiques sont les colonnes
Donc le max sera le max de la colonne
Pour chaque observation,on a plusieurs caractéristiques et la dernière caractéristique est la classe
'''
'''
On évitera d'écrire data.iloc[:,-1]=data.iloc[:,-1].replace({unique_values[0]: 1, unique_values[1]: -1}) ou data.loc[:,data.columns[-1]]=data.loc[:,data.columns[-1]].replace({unique_values[0]: 1, unique_values[1]: -1}) 
pour ne pas avoir l'avertissement de dépréciation, qui indique que dans une future version de pandas, l'utilisation de la méthode iloc,loc
avec une affectation directe ne sera plus recommandée.
'''

def dataFrame_to_collection(data):
    collections = []
    
    for i in range(data.shape[0]):
        unique_values =data.iloc[:,-1].unique()#ici c'est que 2 valeurs unique
      
        data[data.columns[-1]] = data[data.columns[-1]].replace({unique_values[0]: 1, unique_values[1]: -1})
        data = data.apply(pd.to_numeric, errors ='coerce')
        ligne_collections=data.iloc[i].tolist()
       
        collections.append(ligne_collections)

    return np.array(collections)

data_cancer=pd.read_table('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data', sep=',', header=None)
data_spam=pd.read_table('https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data', sep=',', header=None)
data_ionosphere=pd.read_table('https://archive.ics.uci.edu/ml/machine-learning-databases/ionosphere/ionosphere.data', sep=',', header=None)

collection_cancer=dataFrame_to_collection(data_cancer)

collection_spam=dataFrame_to_collection(data_spam)

collection_ionosphere=dataFrame_to_collection(data_ionosphere)


def normalisation(collection):
    # Stratégie max
    collection_max = collection.copy()
    #collection.shape[1]-1 car on ne veut pas normaliser la dernière colonne qui est la classe
    for i in range(collection.shape[1]-1):
        #Pour chaque colonne,on calcule le max
        max= np.max(collection[:,i])
        if(max!=0):
            collection_max[:,i] = collection_max[:,i] / max
    # Stratégie norme
    collection_norme = collection.copy()
    for i in range(collection.shape[0]):
        vecteur=collection[i,:-1]
        norm = np.linalg.norm(vecteur)
        if(norm!=0):
            collection_norme[i,:-1] = collection_norme[i,:-1] / norm
    return collection_max, collection_norme

collection_cancer_max, collection_cancer_norme = normalisation(collection_cancer)
collection_spam_max, collection_spam_norme = normalisation(collection_spam)
collection_ionosphere_max, collection_ionosphere_norme = normalisation(collection_ionosphere)


<font color='red'><b>Question 8:</b></font> Compléter les tableaux comparatifs suivants en repertant les erreurs moyennes sur 20 lancements des modèles de Perceptron, de l'Adaline, de la Régression Logistique et des SVM pour les trois cas:

 '*' Les vecteurs ne sont pas normalisés
     
  | Collection | Adaline     | Perceptron  | Régression Logistique | SVM |
  |------------|-------------|-------------|-----------------------|-----|
  |   BREAST   |             |             |                       |     |
  |   IONO     |             |             |                       |     |
  |   SONAR    |             |             |                       |     |
  |   SPAM     |             |             |                       |     |

 
 $^n$ Normalisation suivant la stratégie <i>norme</i>
     
  | Collection | Adaline     | Perceptron  | Régression Logistique | SVM |
  |------------|-------------|-------------|-----------------------|-----|
  |   BREAST   |             |             |                       |     |
  |   IONO     |             |             |                       |     |
  |   SONAR    |             |             |                       |     |
  |   SPAM     |             |             |                       |     |

  
 $^m$ Normalisation suivant la stratégie <i>max</i>
    
  | Collection | Adaline     | Perceptron  | Régression Logistique | SVM |
  |------------|-------------|-------------|-----------------------|-----|
  |   BREAST   |             |             |                       |     |
  |   IONO     |             |             |                       |     |
  |   SONAR    |             |             |                       |     |
  |   SPAM     |             |             |                       |     |


In [109]:

def train_models(collection):
    perceptron = Perceptron(len(collection[0]), eta)
    adaline = Adaline(len(collection[0]), eta)
    regression_logistique = LogisticRegression(len(collection[0]), eta)
    svm= SVM(len(collection[0]), eta)
    return perceptron.fit(collection_cancer,collection_cancer.shape[0],MaxEp), adaline.fit(collection_cancer,collection_cancer.shape[0],MaxEp),regression_logistique.fit(collection_cancer,collection_cancer.shape[0],MaxEp), svm.fit(collection_cancer,collection_cancer.shape[0],MaxEp)

for j in range(3):
    
    XX = collection_cancer[:,:-1]
    YY = collection_cancer[:,-1]
    erreur_p =[]
    erreur_a =[]
    erreur_r =[]
    erreur_s =[]
    nb_base_tests=20
    table = [[],[],[],[]]

    #A chaque itéaration,on aura des bases d'entrainement et de test différentes
    for i in range(nb_base_tests):
        #Générer une base d'entrainement et une base de test de façon aléatoire grace a train_test_split
        X_train, X_test, Y_train, Y_test = train_test_split(XX, YY, test_size=0.25)
        
        Test= np.column_stack((X_test, Y_test))
        Train= np.column_stack((X_train, Y_train))

        w_p=np.zeros(collection_cancer.shape[1])
        w_a=np.zeros(collection_cancer.shape[1])
        w_r=np.zeros(collection_cancer.shape[1])
        w_s=np.zeros(collection_cancer.shape[1])

       
        if(j==0):
            w_p, w_a, w_r, w_s = train_models(collection_cancer)    
        elif(j==1):
            w_p, w_a, w_r, w_s = train_models(collection_cancer_norme)
        elif (j==2):
            w_p, w_a, w_r, w_s = train_models(collection_cancer_max)
        
        
        # Calculer l'erreur moyenne sur les données de test
        erreur_p.append(perceptron.TAUX_ERREUR(Test))
        erreur_a.append(adaline.TAUX_ERREUR(Test))
        erreur_r.append(regression_logistique.TAUX_ERREUR(Test))
        erreur_s.append(svm.TAUX_ERREUR(Test))
    if(j==0):   
        print("Not normalized version ")  
    elif(j==1):
        print("Normalized version ") 
    elif (j==2):
        print("Max version ") 
    print("Adaline", np.mean(erreur_a),"Perceptron ", np.mean(erreur_p), "regression logistique", np.mean(erreur_r), "SVM", np.mean(erreur_s))


/tmp/ipykernel_13910/1455021576.py:32: RuntimeWarning: overflow encountered in exp
  L=(1-(1/(1+np.exp(-y*self.h(x)))))


ValueError: shapes (10,) and (60,) not aligned: 10 (dim 0) != 60 (dim 0)